## **Final publication charts for Britain's Disappearing Dynamism**

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import altair as alt
import eco_style 

os.chdir("CEP Dynamism Project")

In [ ]:
# Load data


#### **Chart 1 - The UK's evaporating productivity growth**

Output per hour worked between 1973 - 2025. This has been rebased to the four quarters average in 2008.

Data source: https://www.ons.gov.uk/economy/economicoutputandproductivity/productivitymeasures/datasets/outputperhourworkeduk

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

# Load and prepare data
df = pd.read_csv('output_per_hour_worked_ons_dec25.csv', skiprows=6)
df = df.rename(columns={df.columns[0]: 'year', df.columns[1]: 'output_per_hour_worked'})

# Convert "1971 Q1" format to datetime
def quarter_to_date(quarter_str):
    """Convert '1971 Q1' to '1971-01-01' (start of quarter)"""
    year, quarter = quarter_str.split()
    quarter_num = int(quarter[1])  # Extract number from 'Q1', 'Q2', etc.
    
    # Map quarter to month (start of each quarter)
    month_map = {1: 1, 2: 4, 3: 7, 4: 10}
    month = month_map[quarter_num]
    
    return pd.Timestamp(year=int(year), month=month, day=1)

df['date'] = df['year'].apply(quarter_to_date)

# Rebase to 2008 = 100
value_2008 = df[df['date'].dt.year == 2008]['output_per_hour_worked'].mean()
print(f"2008 base value: {value_2008:.2f}")

df['value'] = (df['output_per_hour_worked'] / value_2008) * 100

print(f"Data range: {df['date'].min().date()} to {df['date'].max().date()}")

# Sort data by date to ensure smooth lines
df = df.sort_values('date')

# Calculate pre-recession trend
recession_start = pd.to_datetime('2008-01-01')
pre_recession = df[df['date'] < recession_start].copy()

# Use the average value at 2008 as the baseline
value_2008_actual = df[df['date'].dt.year == 2008]['value'].mean()

# Fit trend to pre-2008 data
pre_recession['time'] = (pre_recession['date'] - pre_recession['date'].min()).dt.days
z = np.polyfit(pre_recession['time'], pre_recession['value'], 1)

# Calculate what the trend value would be at 2008
time_at_2008 = (recession_start - pre_recession['date'].min()).days
trend_value_at_2008 = np.polyval(z, time_at_2008)

# Adjust the trend to pass through the actual 2008 value
# Shift the intercept so trend matches actual at 2008
adjustment = value_2008_actual - trend_value_at_2008
z_adjusted = [z[0], z[1] + adjustment]

# Extend adjusted trend to all dates
df['time'] = (df['date'] - pre_recession['date'].min()).dt.days
df['trend'] = np.polyval(z_adjusted, df['time'])

# Create base chart
base = alt.Chart(df).encode(
    x=alt.X('year(date):T', 
            title='', 
            axis=alt.Axis(
                grid=True, 
                gridDash=[2, 2], 
                gridOpacity=0.4,
                tickCount=10
            ))
)

# Actual productivity line (blue)
actual_line = alt.Chart(df).mark_line(
    color='#5B9BD5',
    strokeWidth=2.5
).encode(
    x=alt.X('date:T', 
            title='', 
            axis=alt.Axis(
                grid=True, 
                gridDash=[2, 2], 
                gridOpacity=0.4,
                tickCount=10,
                format='%Y'
            )),
    y=alt.Y('value:Q', 
            title='', 
            scale=alt.Scale(domain=[0, 160]),
            axis=alt.Axis(
                grid=True, 
                gridDash=[2, 2], 
                gridOpacity=0.4
            ))
)

# Pre-recession trend line (red dashed) - show for entire period
trend_line = alt.Chart(df).mark_line(
    color='#C94545',
    strokeDash=[8, 4],
    strokeWidth=2.5,
    interpolate='natural'
).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(format='%Y')),
    y='trend:Q'
)

# Vertical line at recession start
recession_line = alt.Chart(pd.DataFrame({
    'x': [recession_start]
})).mark_rule(
    strokeDash=[6, 4],
    strokeWidth=1.5,
    color='#808080'
).encode(
    x=alt.X('x:T', title='')
)

# Label: "Pre-recession trend"
trend_label_date = pd.to_datetime('2016-01-01')
trend_value_at_label = df[df['date'] == trend_label_date]['trend'].values[0] if len(df[df['date'] == trend_label_date]) > 0 else 115

trend_label = alt.Chart(pd.DataFrame({
    'x': [trend_label_date],
    'y': [trend_value_at_label + 14],  # Position above the trend line
    'text': ['Pre-recession trend']
})).mark_text(
    align='center',
    color='#C94545',
    fontSize=12
).encode(
    x=alt.X('x:T', title=''),
    y='y:Q',
    text='text:N'
)

# Label: "Productivity gap" 
latest_date = df['date'].max()
latest_actual = df[df['date'] == latest_date]['value'].values[0]
latest_trend = df[df['date'] == latest_date]['trend'].values[0]

gap_label = alt.Chart(pd.DataFrame({
    'x': [latest_date + pd.DateOffset(years=1)],
    'y': [latest_trend - 8],
    'text': ['Productivity gap']
})).mark_text(
    align='left',
    color='#5B9BD5',
    fontSize=12
).encode(
    x=alt.X('x:T', title=''),
    y='y:Q',
    text='text:N'
)

# Line showing the gap with arrows
gap_line = alt.Chart(pd.DataFrame({
    'x': [latest_date],
    'y': [latest_actual],
    'y2': [latest_trend]
})).mark_rule(
    color='#5B9BD5',
    strokeWidth=2,
    strokeDash=[3, 3]
).encode(
    x=alt.X('x:T', title=''),
    y='y:Q',
    y2='y2:Q'
)

# Arrow markers using triangle shapes
# Top arrow (pointing down at trend line)
gap_arrow_top = alt.Chart(pd.DataFrame({
    'x': [latest_date],
    'y': [latest_trend - 1]  # Slightly below the trend line
})).mark_point(
    shape='triangle-up',
    filled=True,
    color='#5B9BD5',
    size=100
).encode(
    x=alt.X('x:T', title='', axis=alt.Axis(format='%Y')),
    y='y:Q'
)

# Bottom arrow (pointing up at actual line)  
gap_arrow_bottom = alt.Chart(pd.DataFrame({
    'x': [latest_date],
    'y': [latest_actual + 1]  # Slightly above the actual line
})).mark_point(
    shape='triangle-down',
    filled=True,
    color='#5B9BD5',
    size=100
).encode(
    x=alt.X('x:T', title='', axis=alt.Axis(format='%Y')),
    y='y:Q'
)

# Combine all layers
chart = (
    recession_line + 
    actual_line + 
    trend_line + 
    trend_label + 
    gap_label +
    gap_line +
    gap_arrow_top +
    gap_arrow_bottom
).properties(
    width=600,
    height=400

).configure_view(
    strokeWidth=0,
    stroke=None
).configure_axis(
    gridColor='#D3D3D3',
    domainColor='#000000',
    tickColor='#000000'
)

# Save and display
chart.save('Paper charts/productivity_chart.png', scale_factor=2)
chart.save('Paper charts/productivity_chart.json')
print("✓ Chart saved as 'productivity_chart.html'")
print(f"✓ Latest productivity (2008=100): {latest_actual:.1f}")
print(f"✓ Trend projection: {latest_trend:.1f}")
print(f"✓ Productivity gap: {latest_trend - latest_actual:.1f} points")

chart

## **Chart 2 - BSD basic facts**